In [20]:
import pandas as pd
from google import genai
from google.genai import types
import unidecode  


# Limpieza

In [21]:
data = pd.read_csv('data/encuesta_raw.csv')
# remover respuesta de prueba
data = data[~(data['Id'] == 1)]
# remover columnas innecesaias
data.drop(['Email', 'Name'], axis=1, inplace=True)

# renombrar las columnas
data.rename(columns={
    'Id': 'response_id',
    'Start time': 'start_response',
    'Completion time': 'complete_response',
    'Edad': 'age',
    'Sexo': 'sex',
    'Año de estudio en el área de la salud.': 'years_studying',
    'Que carrera esta cursando?': 'degree',
    '¿Cómo calificaría el estado actual de la infraestructura hospitalaria en Managua?': 'infrastructure_score',
    '¿Ha notado mejoras en la infraestructura hospitalaria durante el período 2020–2025?': 'notice_infrastructure_improvement',
    'Mencione almenos 3 cambios en la infraestructura hospitalaria que considere relevante que haya llevado a cabo nuestro gobierno revolucionario.': 'infrastructure_changes_raw',
    '¿Con qué frecuencia ha observado el uso de  sistemas digitales (expedientes electrónicos, telemedicina, imágenes  digitales) durante sus prácticas o rotaciones?': 'frecuency_digital_systems',
    '¿Cree que la incorporación de nuevas tecnologías ha mejorado la atención a los pacientes?': 'technology_improves_attention',
    'Mencione una tecnología hospitalaria que considere más importante en la actualidad.': 'most_important_technology_raw',
    'En su experiencia, ¿la modernización hospitalaria ha mejorado la calidad de la atención?': 'modernization_improves_attention',
    '¿Qué aspecto considera que ha mejorado más? ': 'improved_aspect_raw',
    '¿Qué desafíos persisten en la atención hospitalaria a pesar de la modernización?': 'challenges_raw',
    '¿Cuáles son las patologías más frecuentes que ha observado en sus prácticas?': 'pathologies_raw',
    '¿Considera que la red hospitalaria está mejor preparada para atender estas patologías en comparación con hace 5 años?': 'improved_pathologies_treatments',
    'En general, ¿cómo calificaría la modernización de la red hospitalaria pública en Managua (2020–2025)?  ': 'modernization_score',
    '¿Qué recomendación daría para mejorar la modernización hospitalaria en Nicaragua?': 'recomendations_raw'
}, inplace=True)

data['degree'] = data['degree'].replace(
    'Doctor en Medicina y Cirugía', 'Medicina y Cirugía')


# Normalizar el texto de las variables abiertas
def normalize_text(text: str):
    if pd.isna(text):
        return text
    text = unidecode.unidecode(text.replace('ñ', 'ni'))
    text = text.lower().strip()
    text = ' '.join(text.split())
    return text


data["infrastructure_changes_raw"] = data["infrastructure_changes_raw"].apply(
    normalize_text)
data["most_important_technology_raw"] = data["most_important_technology_raw"].apply(
    normalize_text)
data["improved_aspect_raw"] = data["improved_aspect_raw"].apply(normalize_text)
data["challenges_raw"] = data["challenges_raw"].apply(normalize_text)
data["pathologies_raw"] = data["pathologies_raw"].apply(normalize_text)
data["recomendations_raw"] = data["pathologies_raw"].apply(normalize_text)


# corregir tipos de datos
data = data.astype({
    'response_id': 'string',
    'start_response': 'datetime64[ns]',
    'complete_response': 'datetime64[ns]',
    'age': 'category',
    'sex': 'category',
    'years_studying': 'category',
    'degree': 'category',
    'infrastructure_score': 'category',
    'notice_infrastructure_improvement': 'category',
    'infrastructure_changes_raw': 'string',
    'frecuency_digital_systems': 'category',
    'technology_improves_attention': 'category',
    'most_important_technology_raw': 'string',
    'modernization_improves_attention': 'category',
    'improved_aspect_raw': 'string',
    'challenges_raw': 'string',
    'pathologies_raw': 'string',
    'improved_pathologies_treatments': 'category',
    'modernization_score': 'int64',
    'recomendations_raw': 'string'

})

data.info()

<class 'pandas.core.frame.DataFrame'>
Index: 109 entries, 1 to 109
Data columns (total 20 columns):
 #   Column                             Non-Null Count  Dtype         
---  ------                             --------------  -----         
 0   response_id                        109 non-null    string        
 1   start_response                     109 non-null    datetime64[ns]
 2   complete_response                  109 non-null    datetime64[ns]
 3   age                                109 non-null    category      
 4   sex                                109 non-null    category      
 5   years_studying                     109 non-null    category      
 6   degree                             109 non-null    category      
 7   infrastructure_score               109 non-null    category      
 8   notice_infrastructure_improvement  109 non-null    category      
 9   infrastructure_changes_raw         109 non-null    string        
 10  frecuency_digital_systems          109 non-

## Agregar categorias a las carreras

In [22]:
DEGREE_CATEGORY_MAP = {
    # Personal Médico y Quirúrgico
    "Medicina y Cirugía": "Personal Médico y Quirúrgico",
    "Licenciatura en Anestesia y Reanimación": "Personal Médico y Quirúrgico",

    # Personal de Enfermería
    "Licenciatura en Enfermería": "Personal de Enfermería",

    # Personal de Diagnóstico y Laboratorio
    "Licenciatura en Bioanálisis Clínico": "Personal de Diagnóstico y Laboratorio",
    "Licenciatura en Microbiología": "Personal de Diagnóstico y Laboratorio",
    "Técnico Superior en Citología Cervical": "Personal de Diagnóstico y Laboratorio",

    # Personal de Terapia y Rehabilitación
    "Licenciatura en Fisioterapia": "Personal de Terapia y Rehabilitación",
    "Licenciatura en Nutrición": "Personal de Terapia y Rehabilitación",

    # Personal de Atención Especializada
    "Odontología": "Personal de Atención Especializada",
    "Licenciatura en Optometría Médica": "Personal de Atención Especializada",
    "Podología": "Personal de Atención Especializada",

    # Personal de Salud Pública y Prevención
    "Técnico Superior en Higiene y Epidemiología": "Personal de Salud Pública y Prevención"
}


data['degree_category'] = data['degree'].apply(
    lambda x: DEGREE_CATEGORY_MAP[x]).astype('category')

# Limpiar campos de preguntas abiertas

In [25]:
from pydantic import BaseModel, Field
from typing import List
import json

class Answer(BaseModel):
    items: list[str] = Field(..., description="A list of items extracted from the answer. if the answer is not valid, then it should be an empty list")
    rejection_reason: str | None = Field(description="If the answer is invalid, provide a brief reason (e.g., 'Off-topic', 'Spam').")

class BatchFormattedResponse(BaseModel):
    formatted_results: List[Answer] = Field(
        ..., 
        description="A list of the formatted answers, one for each input item, in the same order."
    )


def clean_unstructed_text(question: str, answers: list[str]):
    client = genai.Client()

    prompt = f"""
    You will be given a JSON list of user responses.
    Your task is to format each one according to these rules:
    1. Parse answers for this question "{question}"
    2. A valid response to a question can be a paragraph explaining or a directly any kind of list.
    4. Return *only* the JSON object matching the required schema.
    3. For each element of the main list:
        1. Evaluate if it is a valid response to the question and if not leave the list empty.
        6. The content of each output item can not contain special characters and must be plain text.
        7. Each item should be reduce to a compact version that still expresses a valid response item for the question.
        8. If the answer is invalid, give a breve explanation for why it is not a valid answer.

    Answers:
    {json.dumps(answers)}
    """
    response = client.models.generate_content(
                                           model='gemini-2.5-flash',
                                           contents=prompt,
                                           config=types.GenerateContentConfig(
                                               thinking_config=types.ThinkingConfig(
                                                   thinking_budget=0),
                                            response_mime_type='application/json',
                                            response_json_schema=BatchFormattedResponse.model_json_schema()
                                           ),

                                        )
    response_df = pd.DataFrame(BatchFormattedResponse.model_validate_json(response.text).model_dump()['formatted_results'])

    return response_df



In [ ]:
question = "Mencione cambios en la infraestructura hospitalaria que considere relevante"
result_df = clean_unstructed_text(question, data['infrastructure_changes_raw'].to_list())
result_df.columns = ['infrastructure_changes', 'infrastructure_changes_rejection_reason']
data[result_df.columns] = result_df.values

In [28]:
data[["infrastructure_changes", "infrastructure_changes_rejection_reason"]]

,infrastructure_changes,infrastructure_changes_rejection_reason
1,[atencion],None
2,[],off-topic
3,"[banios, salas de espera, salas de enfermos]",None
4,[sala hospitalaria solo para personas oncologi...,None
5,[mejor comodidad del paciente],None
...,...,...
105,"[ampliacion del area de emergencia en el HAN, ...",None
106,"[dar practicas desde primer anio, maquinas de ...",None
107,"[contratar, capacitar y dar prioridad al perso...",None
108,"[nuevos edificios hospitalarios, nuevas salas ...",None


# Save data

In [29]:
data.to_parquet('data/encuesta.parquet')